Structure of the paper will be as follows...

R2 - IS and OOS, boostrap errors.

New Loss function v random - IS and OOS, boostrap errors.

### 0.0 Inputs...

In [100]:
from __future__ import division, print_function

import joblib
import numpy as np
from IPython.display import SVG
from scipy.spatial.distance import pdist, squareform, jaccard, cityblock
from scipy import stats

from multiprocessing import Pool
from copy import deepcopy

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model as LM

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, RidgeCV, BayesianRidge, ElasticNet, Lasso

from keras.models import Sequential
from keras.layers import Dense, Activation

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.svm import SVR

# Deep learning model with intermediate layer...
def larger_model():
    # create model
    model = Sequential()
    model.add(Dense(128, input_dim=128, kernel_initializer='normal', activation='relu'))
    model.add(Dense(16, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=larger_model, epochs=100, batch_size=5, verbose=0)))

# Model dictionaries 
model_dict = {'ridge' : {'m' : Ridge, 'kw' : {'fit_intercept':True, 'alpha':0.1}, },
              'rf'    : {'m' : RandomForestRegressor, 'kw' : {'n_estimators':100, 
                                                              'n_jobs':4, 'max_depth':10}, },
              'svr'   : {'m' : SVR, 'kw' : {}, },
              
              'dl_l'   : {'m' : Pipeline, 
                          'kw' : {'steps': [('standardize', StandardScaler()),
                                            ('mlp', KerasRegressor(build_fn=larger_model, 
                                                                   epochs=100, batch_size=5, 
                                                                   verbose=0))
                                                           ]},
                         },
             }

# Datasets
from glob import glob
targets = [s.replace('datasets/', '') for s in glob('datasets/*')]

import matplotlib.pyplot as plt
%pylab inline
figsize(20, 10)
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [101]:
# Take out when we want to run deep learning model
#del model_dict['dl_l']

In [102]:
# Syntax testing...
#del model_dict['rf']
#del model_dict['svr']

In [103]:
# Pull in data for a single target name
def get_data(tgt_name='COX-2'):
    data_dir = 'datasets/' + tgt_name + '/'
    preds = joblib.load(data_dir + tgt_name + '_predsu.npy')
    resps = joblib.load(data_dir + tgt_name + '_respu.npy')
    smiles = joblib.load(data_dir + tgt_name + '.smiu')
    return preds, resps, smiles

In [104]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def avg_mse(predictions, responses, **kwargs):
    return mean_squared_error(responses, predictions) / mean_squared_error(responses, np.zeros_like(responses))


def james_loss(predictions, responses, **kwargs):
    
    tgt_val = kwargs.get('tgt_val')
    ranked = np.argsort(-predictions)
    found = responses[ranked] >= tgt_val
    # Number of actives
    N_gamma = np.sum(found)
    
    # Size of test sets
    N_test = predictions.shape[0]
    lt = kwargs.get('loss_type')
    #pdb.set_trace()
    if lt == 'min':
        # Equation (1) of the paper
        loss = 1/(N_test - N_gamma) * np.min(np.arange(N_test)[found])
    elif lt == 'avg':
        # Equation (2) of the paper
        loss = 1/N_gamma * 1/(N_test - N_gamma) * (np.sum(np.arange(N_test)[found]) - N_gamma * (N_gamma - 1)/2)
        pass
    
    assert loss >= 0
    assert loss <= 1
    return loss


def james_loss_avg(predictions, responses, **kwargs):
    kwargs.update({'loss_type' :'avg'})
    return james_loss(predictions, responses, **kwargs)

def james_loss_min(predictions, responses, **kwargs):
    kwargs.update({'loss_type' :'min'})
    return james_loss(predictions, responses, **kwargs)


def nbs_run(kwargs):
    my_is = kwargs.get('is')
    my_oos = kwargs.get('oos')
    method = kwargs.get('method')
    preds = kwargs.get('preds') + 0.
    resps = kwargs.get('resps')
    
    mdl = model_dict[method]['m'](**model_dict[method]['kw'])
    mdl.fit(preds[my_is], resps[my_is])
        
    predictions = mdl.predict(preds[my_oos])
    
    losses = {}
    for (l, v) in loss_dict.iteritems():
        
        # Worth noting, we're looking for the to frac_find _in_the_oos_data_
        # (not in the whole data - as otherwise we might be looking for something
        # that isn't there)
        if 'frac_find' in v['kw']:
            N = len(resps[my_oos])
            sorted_indices = np.argsort(resps[my_oos])
            n = int(N * v['kw']['frac_find'])
            tgt_val = resps[my_oos][sorted_indices[n]]
            v['kw'].update({'tgt_val' : tgt_val})
            
        losses[l] = v['func'](predictions, resps[my_oos], **v['kw'])
        pass
    
    return losses
    
    
def full_bootstrap(preds, resps, method, num_runs=100, insample=False,
                  frac_fit=1.0, num_kf=0, use_pool=True):

    # losses is going to be a list of dicts, loss_type => value
    losses = []
    sorted_indices = np.argsort(resps)

    N = len(resps)
    M = int(N * frac_fit)
    
    # Choose insample values...
    idx_list = []
    for i in range(num_runs):
        if num_kf > 0:
            kf = KFold(n_splits=num_kf, shuffle=True)
            for (tr_i, tst_i) in kf.split(np.arange(M)):
                idcs = sorted_indices[tr_i]
                pass
            pass
        else:
            idcs_rand = np.random.choice(M, M)
            idcs = sorted_indices[idcs_rand]
            pass
        idx_list.append({'is' : idcs})
        pass
    
    # Add in all the other data...
    for d in idx_list:
        if insample:
            oos = d['is']
        else:
            oos = np.delete(np.arange(N), d['is'])
            pass
        d.update({'oos' : oos, 'preds' : preds, 
                  'resps' : resps, 'method' : method})
        pass
            
    if use_pool:
        p = Pool(5)
        losses = p.map(nbs_run, idx_list)
        p.close()
        p.join()
    else:
        losses = [nbs_run(x) for x in idx_list]
        pass
    return losses

In [105]:
# Unit tests for the loss functions...
responses = np.arange(10)
tgt_val = 8

# get it totally right
predictions = np.arange(10)

print('All correct')
print('r2', avg_mse(predictions, responses, tgt_val=tgt_val))
print('james min', james_loss_min(predictions, responses, tgt_val=tgt_val))
print('james avg', james_loss_avg(predictions, responses, tgt_val=tgt_val))

# get it totally right where it counts, but maximally wrong elsewhere
#predictions = np.asarray([0])

# one totally right, one totally wrong
predictions = np.roll(np.arange(10),-1)
print('\nOne right, one very wrong')
print('r2', avg_mse(predictions, responses, tgt_val=tgt_val))
print('james min', james_loss_min(predictions, responses, tgt_val=tgt_val))
print('james avg', james_loss_avg(predictions, responses, tgt_val=tgt_val))


# get everything wrong
predictions = np.arange(10)[::-1]
print('\nAll wrong')
print('r2', avg_mse(predictions, responses, tgt_val=tgt_val))
print('james min', james_loss_min(predictions, responses, tgt_val=tgt_val))
print('james avg', james_loss_avg(predictions, responses, tgt_val=tgt_val))
# 

All correct
r2 0.0
james min 0.0
james avg 0.0

One right, one very wrong
r2 0.315789473684
james min 0.0
james avg 0.5

All wrong
r2 1.15789473684
james min 1.0
james avg 1.0


In [106]:
def summarize(results):
    summary = {}
    for l in loss_dict.keys():
        vals = np.asarray([x[l] for x in results])
        if len(vals) == 1:
            summary[l] = {'loss' : vals[0]}
        else:
            summary[l] = {'loss_l' : np.percentile(vals, 5),
                          'loss' : np.mean(vals),                          
                          'loss_u' : np.percentile(vals, 95),
                         }
            pass
        pass
    return summary

In [107]:
sorted_targets = sorted(targets, key=lambda x: len(get_data(x)[0]))

In [108]:
import os.path

loss_dict = {'mse' :       {'func' : avg_mse,         'kw' : {}},
             'jmin_90' :   {'func' : james_loss_min,  'kw' : {'frac_find' : 0.9}},
             'javg_90' :   {'func' : james_loss_avg,  'kw' : {'frac_find' : 0.9}},
             'jmin_95' :   {'func' : james_loss_min,  'kw' : {'frac_find' : 0.95}},
             'javg_95' :   {'func' : james_loss_avg,  'kw' : {'frac_find' : 0.95}},
             'jmin_99' :   {'func' : james_loss_min,  'kw' : {'frac_find' : 0.99}},
             'javg_99' :   {'func' : james_loss_avg,  'kw' : {'frac_find' : 0.99}},
                           }
def get_fn(frac_fit, kf=0, insample=False):
    fn = 'loss_' + str(frac_fit)
    if kf > 0:
        fn += '_kf_' + str(kf)
        pass
    if insample:
        fn += '_insample'
        pass
    return os.path.join('models_final_dl', fn)

def runner(frac_fit=1.0, kf=0, insample=False, use_pool=True):

    fnf = get_fn(frac_fit, kf=kf, insample=insample)
    if os.path.isfile(fnf) and not force_rerun:
        print('Already computed')
        return

    loss_hdr = {}
    if insample:
        num_runs = 1
    elif kf > 0:
        num_runs = int(200/kf)
    else:
        num_runs = 200
    
    for tgt in sorted_targets:
        loss_hdr[tgt] = {}
        print ('Doing', tgt)
        preds, resps, _ = get_data(tgt)
        preds = preds + 0.
        for m in model_dict.keys():
            res = full_bootstrap(preds, resps, m, frac_fit=frac_fit, num_kf=kf,
                                 insample=insample, num_runs=num_runs, use_pool=use_pool,
                                )
            loss_hdr[tgt][m] = summarize(res)
        pass
    joblib.dump(loss_hdr, fnf)
    print('Completed')
    return

In [109]:
%pylab inline
figsize(20, 10)
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [110]:
force_rerun = True

In [ ]:
# insample run...
runner(insample=True)

Doing A2a
Doing Dopamine
Doing Dihydrofolate
Doing Carbonic
Doing ABL1
Doing opioid
Doing Cannabinoid
Doing Androgen
Doing COX-1
Doing Monoamine
Doing LCK
Doing Glucocorticoid
Doing Ephrin
Doing Caspase
Doing Coagulation
Doing Estrogen
Doing B-raf
Doing Glycogen
Doing Vanilloid
Doing Aurora-A
Doing JAK2
Doing COX-2
Doing Acetylcholinesterase
Doing erbB1
Doing HERG
Completed


In [ ]:
# Full OOS run...
runner()

Doing A2a
Doing Dopamine
Doing Dihydrofolate
Doing Carbonic
Doing ABL1
Doing opioid
Doing Cannabinoid
Doing Androgen
Doing COX-1
Doing Monoamine
Doing LCK
Doing Glucocorticoid


Process PoolWorker-4236:
Process PoolWorker-4240:
Process PoolWorker-4238:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
    self.run()
    self.run()
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/anaconda2/lib/python2.7/multiprocessing/pool.py", line 113, in worker
    result = (True, func(*args, **kwds))
  File "/anaconda2/lib/python2.7/multiprocessing/pool.py", line 65, in mapstar
    return map(*args)
  File "<ipython-input-104-394ac3e4d1d6>", line 50, in nbs_run
    mdl.fit(preds[my_is], resps[my_is])
  File "/anaconda2/lib/python2.7/site-packages/sklearn/pipeline.py", line 250, in fit
    

In [ ]:
# OOS run with 5-fold CV for comparison
runner(kf=5)

In [ ]:
# OOS - max activity at 0.9
runner(frac_fit=0.9)

In [ ]:
# OOS - max activity at 0.8
runner(frac_fit=0.8)

In [ ]:
# OOS - max activity at 0.6
runner(frac_fit=0.6)

In [ ]:
# OOS - max activity at 0.4
runner(frac_fit=0.4)

In [ ]:
def plotter(frac_fit=1.0, kf=0, insample=False, loss='mse'):
    
    fnf = get_fn(frac_fit, kf=0, insample=insample)
    
    loss_hdr = joblib.load(fnf)
    
    for (i, method) in enumerate(model_dict.keys()):
        losses = np.asarray([loss_hdr[x][method][loss]['loss'] for x in sorted_targets])
        if insample:
            plot(np.arange(25) + 0.05*i, losses,  label=method.upper())
        else:
            loss_l = np.asarray([loss_hdr[x][method][loss]['loss_l'] for x in sorted_targets])
            loss_u = np.asarray([loss_hdr[x][method][loss]['loss_u'] for x in sorted_targets])
            
            yerr = np.vstack((losses - loss_l, loss_u - losses))
            
            errorbar(np.arange(25)+i*0.1, losses, capsize=10, yerr=yerr, label=method.upper())
            pass
        pass
    
    if insample:
        title('Insample %s Loss' % loss.upper())
    else:
        ttl = 'OOS %s Loss' % loss.upper()
        if kf > 0:
            ttl += ' with %d fold CV' % kf
        if frac_fit < 1.0:
            ttl += ' Max activity in fit at %.1f' % frac_fit
        title(ttl)
    pass

    grid(True)
    plt.xticks(np.arange(25), sorted_targets, rotation=-45)
    legend(loc='best')

### Insample loss plots

In [ ]:
plotter(insample=True)

In [ ]:
plotter(insample=True, loss='jmin_90')

In [ ]:
plotter(insample=True, loss='jmin_99')

In [ ]:
plotter(insample=True, loss='javg_90')

### OOS (no fit restriction) loss plots (using CV)

In [ ]:
plotter(kf=5)

In [ ]:
plotter(kf=5, loss='javg_90')

### OOS (no fit restriction) loss plots...

In [ ]:
plotter()

In [ ]:
plotter(loss='javg_90')

In [ ]:
plotter(loss='jmin_90')

### OOS plots (0.9 frac fit)

In [ ]:
plotter(frac_fit=0.9)

In [ ]:
plotter(frac_fit=0.9, loss='javg_90')

In [ ]:
plotter(frac_fit=0.9, loss='jmin_90')

### OOS loss (max activity at 0.8)

In [ ]:
plotter(frac_fit=0.8)

In [ ]:
plotter(frac_fit=0.8, loss='javg_90')

In [ ]:
plotter(frac_fit=0.8, loss='jmin_90')

### OOS loss - max activity 0.6

In [ ]:
plotter(frac_fit=0.6)

In [ ]:
plotter(frac_fit=0.6, loss='javg_90')

In [ ]:
plotter(frac_fit=0.6, loss='jmin_90')

### OOS - max activity in fit 0.4

In [ ]:
plotter(frac_fit=0.4)

In [ ]:
plotter(frac_fit=0.4, loss='javg_90')

In [ ]:
plotter(frac_fit=0.4, loss='jmin_90')

In [ ]:
plotter(frac_fit=0.4, loss='jmin_99')

### LL estimates...

In [ ]:
# Since the estimates have wide tails away from 0, we're going to (as a conservative estimate)
# of relative log likelihood, look at how many (2-lower-standard errors away from 0 the mean is)

def ll_estimate(frac_fit=1.0, losses=['javg_90', 'mse'], kf=0):
    fnf = get_fn(frac_fit, kf=0)
    loss_hdr = joblib.load(fnf)
    
    print(('%9s |' + '%9s |' * len(losses)) % tuple(['',] + losses))
    
    print('-' * (11 * (len(losses) + 1) - 1))
    for (i, method) in enumerate(model_dict.keys()):
        
        lls = []
        
        for loss in losses:
            ll = 0
            for tgt in sorted_targets:
                lmean = loss_hdr[tgt][method][loss]['loss']
                llow = loss_hdr[tgt][method][loss]['loss_l']
                sigmas = (lmean - llow)/2
                if lmean == 0:
                    continue # same as subrtacting 0
                else:
                    sigmas_from_zero = lmean/sigmas
                    ll += sigmas_from_zero**2/2
                    pass
                pass
            lls.append(ll)
            pass
        
        print (('%9s |'+ '%9.1f |' * len(losses)) % tuple([method,] + lls))
    
    return

In [ ]:
ll_estimate()

In [ ]:
ll_estimate(frac_fit=0.9)

In [ ]:
ll_estimate(frac_fit=0.8)

In [ ]:
ll_estimate(frac_fit=0.6)

In [ ]:
ll_estimate(frac_fit=0.4)

In [ ]:
ll_estimate(losses=loss_dict.keys(), frac_fit=0.9)

In [ ]:
ll_estimate(losses=loss_dict.keys(), frac_fit=0.8)

In [ ]:
ll_estimate(losses=loss_dict.keys(), frac_fit=0.6)

In [ ]:
ll_estimate(losses=loss_dict.keys(), frac_fit=0.4)